# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [6]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import datetime
import api_token  # handle iexcloud API tokens

env = "sandbox"  # "cloud"
BASE_URL, IEX_CLOUD_API_TOKEN = api_token.get_base_url(env=env), api_token.get_api_token(env=env)

In [7]:
TOP_N = 50
portfolio_size = 1000000

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [8]:
# stocks = pd.read_csv('sp_500_stocks.csv')

df = pd.read_csv('nasdaq_100_stocks.csv.gz', 
                 compression="gzip", 
                 names=["Ticker", "Company"], 
                 skiprows=[0])
stocks = df["Ticker"].values.tolist()

In [9]:
",".join(stocks)

'ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,ANSS,AAPL,AMAT,ASML,ADSK,ADP,BIDU,BIIB,BMRN,BKNG,AVGO,CDNS,CDW,CERN,CHTR,CHKP,CTAS,CSCO,CTXS,CTSH,CMCSA,CPRT,COST,CSX,DXCM,DOCU,DLTR,EBAY,EA,EXC,EXPE,FB,FAST,FISV,FOXA,FOX,GILD,IDXX,ILMN,INCY,INTC,INTU,ISRG,JD,KDP,KLAC,KHC,LRCX,LBTYA,LBTYK,LULU,MAR,MXIM,MELI,MCHP,MU,MSFT,MRNA,MDLZ,MNST,NTES,NFLX,NVDA,NXPI,ORLY,PCAR,PAYX,PYPL,PEP,PDD,QCOM,REGN,ROST,SGEN,SIRI,SWKS,SPLK,SBUX,SNPS,TMUS,TTWO,TSLA,TXN,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,XEL,XLNX,ZM'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [10]:
symbol = 'AAPL'
api_url = f'{BASE_URL}/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2112524571823,
 'week52high': 138.91,
 'week52low': 57,
 'week52change': 0.795939385585816,
 'sharesOutstanding': 17590243114,
 'float': 0,
 'avg10Volume': 90799966,
 'avg30Volume': 95917833,
 'day200MovingAvg': 111.38,
 'day50MovingAvg': 121.02,
 'employees': 0,
 'ttmEPS': 3.4,
 'ttmDividendRate': 0.8129211053819299,
 'dividendYield': 0.006822485800703475,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-31',
 'nextEarningsDate': '0',
 'peRatio': 36.993807010657875,
 'beta': 1.2094134443179418,
 'maxChangePercent': 47.26818022745726,
 'year5ChangePercent': 3.733723968331849,
 'year2ChangePercent': 2.025399392752163,
 'year1ChangePercent': 0.8272806685885513,
 'ytdChangePercent': 0.700638601743302,
 'month6ChangePercent': 0.4636038674584825,
 'month3ChangePercent': 0.0950153530150025,
 'month1ChangePercent': 0.027156644818433027,
 'day30ChangePercent': 0.027270993855999785,
 'day5ChangePercent': 0.0013420259901410563}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [11]:
data['year1ChangePercent']

0.8272806685885513

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [12]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks, 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [13]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,ATVI,83.72,0.440064,N/A
1,ADBE,476.04,0.568678,N/A
2,AMD,92.24,1.162495,N/A
3,ALXN,124.92,0.077344,N/A
4,ALGN,509.53,0.914253,N/A
...,...,...,...,...
98,WBA,42.66,-0.266520,N/A
99,WDAY,219.26,0.382707,N/A
100,XEL,68.00,0.091229,N/A
101,XLNX,145.21,0.520033,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [14]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:(TOP_N+1)]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MRNA,156.950,7.685771,N/A
1,TSLA,611.630,7.521929,N/A
2,ZM,408.100,5.547143,N/A
3,PDD,148.560,2.963366,N/A
4,DOCU,226.390,2.181137,N/A
5,MELI,1618.080,1.790635,N/A
6,JD,83.520,1.512043,N/A
7,NVDA,544.380,1.332223,N/A
8,AMD,92.240,1.162495,N/A
9,PYPL,222.680,1.058369,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [15]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

# portfolio_input()
# print(portfolio_size)

In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/home/wengong/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MRNA,156.950,7.685771,124
1,TSLA,611.630,7.521929,32
2,ZM,408.100,5.547143,48
3,PDD,148.560,2.963366,131
4,DOCU,226.390,2.181137,86
5,MELI,1618.080,1.790635,12
6,JD,83.520,1.512043,234
7,NVDA,544.380,1.332223,36
8,AMD,92.240,1.162495,212
9,PYPL,222.680,1.058369,88


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [17]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [18]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# # Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ATVI,86.27,N/A,0.457807,0.68932,0.168145,0.378641,0.075334,0.339806,0.092607,0.68932,N/A
1,ADBE,493.75,N/A,0.579434,0.776699,0.174332,0.38835,0.009970,0.165049,0.029716,0.38835,N/A
2,AMD,95.85,N/A,1.172212,0.92233,0.726404,0.92233,0.206394,0.718447,0.123018,0.776699,N/A
3,ALXN,122.84,N/A,0.078008,0.252427,0.111562,0.271845,0.148789,0.543689,-0.038617,0.0679612,N/A
4,ALGN,523.28,N/A,0.919923,0.902913,1.044063,0.970874,0.603656,0.951456,0.123909,0.786408,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
98,WBA,42.87,N/A,-0.260393,0.00970874,0.027286,0.126214,0.221467,0.737864,0.031795,0.407767,N/A
99,WDAY,224.51,N/A,0.380010,0.601942,0.267887,0.553398,0.064898,0.291262,-0.002638,0.194175,N/A
100,XEL,67.00,N/A,0.090130,0.300971,0.028947,0.145631,-0.051513,0.106796,-0.120425,0.0291262,N/A
101,XLNX,143.90,N/A,0.513238,0.728155,0.602428,0.912621,0.482824,0.912621,0.127288,0.796117,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [19]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [20]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:(TOP_N+1)]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
# portfolio_input()

In [23]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)

In [24]:
position_size

19607.843137254902

In [29]:
hqm_dataframe.reset_index(drop = True, inplace = True)

In [30]:
len(hqm_dataframe['Ticker'])

51

In [33]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRNA,164.210,119,7.535052,0.990291,1.580231,0.990291,1.711291,1,0.816125,1,0.995146
1,TSLA,638.600,30,7.679719,1,2.280316,1,0.666770,0.970874,0.497633,0.990291,0.990291
2,MELI,1625.690,12,1.841541,0.951456,0.801796,0.951456,0.573289,0.941748,0.246146,0.961165,0.951456
3,LRCX,511.600,38,0.789978,0.854369,0.780714,0.941748,0.713742,0.980583,0.183263,0.941748,0.929612
4,PDD,153.460,127,2.954178,0.970874,1.068642,0.980583,0.744098,0.990291,0.103459,0.728155,0.917476
5,ALGN,523.280,37,0.919923,0.902913,1.044063,0.970874,0.603656,0.951456,0.123909,0.786408,0.902913
6,AMAT,92.700,211,0.491527,0.718447,0.585835,0.902913,0.635950,0.961165,0.271716,0.980583,0.890777
7,XLNX,143.900,136,0.513238,0.728155,0.602428,0.912621,0.482824,0.912621,0.127288,0.796117,0.837379
8,MU,72.680,269,0.393336,0.621359,0.453341,0.815534,0.531440,0.932039,0.262966,0.970874,0.834951
9,AMD,95.850,204,1.172212,0.92233,0.726404,0.92233,0.206394,0.718447,0.123018,0.776699,0.834951


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [34]:
for i in range(len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])

In [35]:
today = datetime.datetime.now()
xlsx_file = f"002-momentum_strategy_{today.strftime('%Y-%m-%d')}.xlsx"

In [36]:
xlsx_file

'002-momentum_strategy_2020-12-13.xlsx'

In [37]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [38]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [39]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [40]:
writer.save()